In [1]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [3]:
column_list = [0,2,12,17,19,41,42,46,51,52,54,56,85,89,90,95]
print('Loading...')

filepath = 'sample-folder/4G_ESS_NBIOT_20230723.xlsb'
title = filepath
filepath = os.path.join(filepath)

# files   = pd.read_csv(filepath, usecols=column_list, low_memory=False)
files = pd.read_excel(filepath, sheet_name='4G', usecols=column_list, dtype=str)

files = files[files['dlChannelBandwidth'].astype(str).str.contains('dlChannelBandwidth')==False].copy(deep=True)
# files = files[files['REMARK'].astype(str).str.contains('FALSE')==True].copy(deep=True) #If using CSV, Boolean dont change
files = files[files['REMARK'].astype(str).str.contains('False')==True] #If using XLSX, string is changed.
files = files[files['operationalState'].astype(str).str.contains('ENABLED')==True]
files = files[files['faultIndicator'].astype(str).str.contains('OFF')==True]

files['BW'] = files['dlChannelBandwidth'].astype(int).div(1000).astype(str)
files['freqBand-NewName'] = files['freqBand'].replace({'8': 'Band-900', '3': 'Band-1800', '1':'Band-2100'}, regex=True)
files['freqBand'] = files['freqBand'].replace({'8': 'Band-900', '3': 'Band-1800', '1':'Band-2100'}, regex=True)

files['TYPE2'].replace({'0':'NO_INFO'}, inplace=True)
files['SECTOR_FINAL'].replace({'x':0}, inplace=True)
files['SECTOR_FINAL'] = files['SECTOR_FINAL'].astype(int).replace({5:4, 6:4}).astype(str)
files['SECTOR_FINAL'] = "SECTOR_FINAL_" + files['SECTOR_FINAL'].astype(str)
files['TYPE2']   = files['TYPE2'].str.upper()

files['freqBand-NewName'] = files['freqBand-NewName'].astype(str).str.strip()
files['productName'].fillna('0', inplace=True)
files['productName'].replace({' ': ''}, inplace=True, regex=True)
files['DATE'] =  pd.to_datetime(files['DATE'])

# ...Sorting the date column as it requires get the most updates rows...
files = files.sort_values(by='DATE', ascending=False)

# ...Create a concatenation of Tower DNK and Column Name to get lookup index...
files['CONCATENATE-1'] = files[['TOWER_DNK', 'freqBand-NewName', 'SECTOR_FINAL']].astype(str).agg('_'.join, axis=1)
files['CONCATENATE-2'] = files[['TOWER_DNK', 'freqBand-NewName', 'serialNumber', 'SECTOR_FINAL']].astype(str).agg('_'.join, axis=1)
files['CONCATENATE-3'] = files[['TOWER_DNK', 'serialNumber', 'SECTOR_FINAL']].astype(str).agg('_'.join, axis=1)
files['CONCATENATE-6'] = files[['TOWER_DNK', 'freqBand', 'sectorFunctionRef']].astype(str).agg('_'.join, axis=1)

files.insert(len(files.columns), column='Qty_productName', value=1)
files['MIMO'] = files['noOfTxAntennas'].astype(str) + "x" + files['noOfRxAntennas'].astype(str)

#...Create 2 files variable : 

# files1 (Files 1 is used for calculate Qty of Radio Per Sector)
# files2 (Files 2 is used for calculate Cell Qty of MIMO and BW)

files1 = files.drop_duplicates(subset=['CONCATENATE-2'], keep='first').copy(deep=True)
files1['CONCATENATE-3_LOGIC'] = files1['CONCATENATE-3'].duplicated(keep=False)
files1.loc[files1['CONCATENATE-3_LOGIC'] == True, 'freqBand-NewName'] = 'Band-1800&2100'
files1['CONCATENATE-1'] = files1[['TOWER_DNK', 'freqBand-NewName', 'SECTOR_FINAL']].astype(str).agg('_'.join, axis=1)
files1['CONCATENATE-1'] = files1[['TYPE2', 'CONCATENATE-1']].astype(str).agg('|'.join, axis=1)
files1['CONCATENATE-2'] = files1[['TOWER_DNK', 'freqBand-NewName', 'serialNumber', 'SECTOR_FINAL']].astype(str).agg('_'.join, axis=1)

#...Drop Duplication as the duplication is a false information ...
files1.drop_duplicates(subset=['CONCATENATE-2'], keep='first', inplace=True)

extract_rf = files1['Rfport'].str.extract(r'(\b)(RfPort=.)')
files1 = files1.join(extract_rf)
files1['CONCATENATE-4'] = files1[['TOWER_DNK', 'serialNumber']].astype(str).agg('_'.join, axis=1)

files2 = files.drop_duplicates(subset=['CONCATENATE-6'], keep='first')

df = pd.DataFrame(files['TOWER_DNK'].unique())
df.rename(columns={df.columns[0]:'TOWER_DNK'}, inplace = True)
df.set_index('TOWER_DNK', inplace=True)

Loading...


In [4]:
#...Flagging which radio has duplication which meaning one radio has been used in other sector... 
files1['DUPLICATED'] = files1['CONCATENATE-4'].duplicated(keep=False)
files1.reset_index(inplace=True)
files1.drop(columns='index', inplace=True)

#...The method is getting the index of duplicated and concatenate them through iloc function...
list1 = list(files1[files1['DUPLICATED'] == True].index)
# files1.iloc[list1, 7] = files1.iloc[list1, 7] + "_" + files1.iloc[list1, 23]
files1.iloc[list1, 8] = files1.iloc[list1, 9] + "_" + files1.iloc[list1, 25]

files1['productName'] = files1[['productName', 'MIMO']].astype(str).agg('_'.join, axis=1)

for a in list(files2['freqBand-NewName'].unique()):
    for b in ['MIMO', 'BW']:    

        df_qty = files2.query(f'["{a}"] in `freqBand-NewName`') 
        df_qty = df_qty.groupby(['TOWER_DNK', b], group_keys=False)['Qty_productName'].sum().reset_index()
        df_qty['COMBINE - 1'] = "[" + df_qty['Qty_productName'].astype(str) + "]" + df_qty[b]

        df_qty = df_qty.groupby(['TOWER_DNK'], group_keys=False)['COMBINE - 1'].apply(lambda x: "%s" % '+'.join(x)).reset_index()
        df_qty.rename(columns={'COMBINE - 1':f'1. VALUE|LTE_RADIO_{a}_{b}'}, inplace=True)
        
        df_qty.set_index('TOWER_DNK', inplace=True)
        df = df.join(df_qty, how='left')
        
for a in list(files1['freqBand-NewName'].unique()):
    for b in list(files1['SECTOR_FINAL'].unique()):
        for c in list(files1['TYPE2'].unique()):

            df_qty = files1.query(f'["{b}"] in SECTOR_FINAL and ["{a}"] in `freqBand-NewName` and ["{c}"] in `TYPE2`')
            df_qty = df_qty.groupby(['TOWER_DNK','CONCATENATE-1', 'productName'], group_keys=False)['Qty_productName'].sum().reset_index()
            df_qty['COMBINE - 1'] = "[" + df_qty['Qty_productName'].astype(str) + "]" + df_qty['productName']
            
            df_qty = df_qty.groupby(['TOWER_DNK'], group_keys=False)['COMBINE - 1'].apply(lambda x: "%s" % '+'.join(x)).reset_index()
            df_qty.rename(columns={'COMBINE - 1':f'{c}|LTE_RADIO_{a}_{b}'}, inplace=True)
            df_qty.set_index('TOWER_DNK', inplace=True)
            df = df.join(df_qty, how='left')

df = df.reset_index()
df.rename(columns={"TOWER_DNK":'Special_Tower_ID_mapping_for_Tower_ID_with_more_than_1_site_ID'}, inplace=True)
df.set_index('Special_Tower_ID_mapping_for_Tower_ID_with_more_than_1_site_ID', inplace=True)     
df.columns = df.columns.str.split('|', expand=True)
df.rename(columns={np.nan : 'value'}, level=1, inplace=True)

for a in df.columns:
    if len(df[a].value_counts()) == 0:
        df.drop(columns=[a], inplace=True)

df.sort_index(axis=1, level=0, inplace=True)

# df.columns = pd.MultiIndex.from_product([1 * [''], df.columns])
df = df.transpose().reset_index()
df = df.transpose()
# df = df.add_suffix(filepath)

title = os.path.split(title)[1]
title = title.split('.')[0]

In [5]:
files = files[['TOWER_DNK', 'FDN']].drop_duplicates(subset='FDN', ignore_index=True)
files['FDN'] = files['FDN'].str.replace(('(') , '').str.replace((')') , '')

files['FDN'] = files['FDN'].str.split(pat='MeContext=', expand=True)[1]
files['FDN'] = files['FDN'].str.split(pat=',', expand=True)[0]

In [6]:
print('Saving...')

with pd.ExcelWriter(f'{title}-RadioSector.xlsx', engine='xlsxwriter') as writer: 
    df.to_excel(writer, header=True, sheet_name='Radio Per Sector')
    files.to_excel(writer, header=True, sheet_name='Mapping-TowerID', index=False)

Saving...
